In [1]:
# Load dataset files with Pandas
import pandas as pd
import os

dataset_path = '../../Datasets/CIC-DDOS2019/Kaggle Preprocessed/'
dataframes = []
file_list = os.listdir(dataset_path)
for file_name in file_list:
    if file_name.endswith('.parquet'):
        file_path = os.path.join(dataset_path, file_name)
        df = pd.read_parquet(file_path)
        dataframes.append(df)

In [2]:
# Concatenate dataset files and correct labels
df = pd.concat(dataframes)
del dataframes
df['Label'] = df['Label'].str.replace('DrDoS_DNS', 'DNS')
df['Label'] = df['Label'].str.replace('DrDoS_LDAP', 'LDAP')
df['Label'] = df['Label'].str.replace('DrDoS_MSSQL', 'MSSQL')
df['Label'] = df['Label'].str.replace('DrDoS_NTP', 'NTP')
df['Label'] = df['Label'].str.replace('DrDoS_NetBIOS', 'NetBIOS')
df['Label'] = df['Label'].str.replace('DrDoS_SNMP', 'SNMP')
df['Label'] = df['Label'].str.replace('DrDoS-SSDP', 'SSDP')
df['Label'] = df['Label'].str.replace('DrDoS_UDP', 'UDP')
df['Label'] = df['Label'].str.replace('UDPLag', 'UDP-lag')

In [ ]:
# Remove classes that have few samples in the graphs approach for conformity
df = df[df['Label'] != 'DNS']
df = df[df['Label'] != 'LDAP']
df = df[df['Label'] != 'MSSQL']
df = df[df['Label'] != 'NTP']
df = df[df['Label'] != 'NetBIOS']
df = df[df['Label'] != 'SNMP']
df = df[df['Label'] != 'SSDP']
df = df[df['Label'] != 'Portmap']
df = df[df['Label'] != 'WebDDoS']

In [3]:
# Split data from labels
data = df[df.columns[:-1]] # type: ignore
categories = df['Label'].astype('category')

In [4]:
# MinMax scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

In [5]:
# 10-fold data (for 10-fold cross validation), 
from sklearn.model_selection import train_test_split
from run_models.run_models import run_models
from imblearn.over_sampling import SMOTE
data_train, data_test, categories_train, categories_test = train_test_split(
    data,
    categories,
    random_state=100,
    test_size=0.1,
    shuffle=True,
    stratify=categories,
)
data_train, data_dev, categories_train, categories_dev = train_test_split(
    data_train,
    categories_train,
    random_state=100,
    test_size=0.1 / (1 - 0.1),
    shuffle=True,
    stratify=categories_train,
)
# Oversample infrequent data using SMOTE
ros = SMOTE(random_state=101)
data_train, categories_train = ros.fit_resample(data_train, categories_train) # type: ignore
results = run_models(data_train, categories_train, data_dev, categories_dev, data_test, categories_test)
print(results)

2023-10-04 23:45:45.259066: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-04 23:45:45.259094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-04 23:45:45.259113: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Fold 1/5

Running MLP Classifier
Epoch 1/20


2023-10-04 23:45:46.870368: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


8628/8628 - 15s - loss: 0.3709 - categorical_crossentropy: 0.3709 - categorical_accuracy: 0.8796 - val_loss: 1.1522 - val_categorical_crossentropy: 1.1522 - val_categorical_accuracy: 0.9197 - 15s/epoch - 2ms/step
Epoch 2/20
8628/8628 - 14s - loss: 0.2390 - categorical_crossentropy: 0.2390 - categorical_accuracy: 0.9102 - val_loss: 1.4590 - val_categorical_crossentropy: 1.4590 - val_categorical_accuracy: 0.9206 - 14s/epoch - 2ms/step
Epoch 3/20
8628/8628 - 14s - loss: 0.2158 - categorical_crossentropy: 0.2158 - categorical_accuracy: 0.9170 - val_loss: 1.7107 - val_categorical_crossentropy: 1.7107 - val_categorical_accuracy: 0.9214 - 14s/epoch - 2ms/step
Epoch 4/20
8628/8628 - 14s - loss: 0.1892 - categorical_crossentropy: 0.1892 - categorical_accuracy: 0.9281 - val_loss: 1.9791 - val_categorical_crossentropy: 1.9791 - val_categorical_accuracy: 0.9224 - 14s/epoch - 2ms/step
2697/2697 [==============================] - 3s 981us/step


/home/greg/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Accuracy: 0.9165913253123769
Macro-Precision: 0.42471198080078926
Macro-Recall: 0.4708535991594758
Macro-F-Score: 0.4070740001172421
               precision    recall  f1-score   support

       Benign       0.99      1.00      0.99     19566
    DrDoS_DNS       0.41      0.27      0.33       734
   DrDoS_LDAP       1.00      0.00      0.00       288
  DrDoS_MSSQL       1.00      0.00      0.00      1243
    DrDoS_NTP       0.99      0.99      0.99     24273
DrDoS_NetBIOS       1.00      0.00      0.00       120
   DrDoS_SNMP       0.36      0.85      0.50       543
    DrDoS_UDP       0.50      0.00      0.00      2084
         LDAP       1.00      0.00      0.00       382
        MSSQL       0.50      0.90      0.64      1704
      NetBIOS       0.30      0.81      0.44       128
      Portmap       0.05      0.01      0.01       137
          Syn       0.91      0.97      0.94      9875
         TFTP       1.00      0.99      1.00     19783
          UDP       0.57      0.96      0

/home/greg/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9132299418132925
Macro-Precision: 0.4678619656511985
Macro-Recall: 0.5017388305116462
Macro-F-Score: 0.4473906249781611
               precision    recall  f1-score   support

       Benign       0.99      1.00      0.99     19566
    DrDoS_DNS       0.34      0.42      0.38       734
   DrDoS_LDAP       1.00      0.00      0.00       288
  DrDoS_MSSQL       1.00      0.00      0.00      1243
    DrDoS_NTP       0.99      0.99      0.99     24273
DrDoS_NetBIOS       0.00      0.00      0.00       120
   DrDoS_SNMP       0.48      0.43      0.45       543
    DrDoS_UDP       0.33      0.96      0.49      2084
         LDAP       0.33      0.45      0.38       382
        MSSQL       0.49      0.91      0.64      1704
      NetBIOS       0.35      0.62      0.45       128
      Portmap       0.00      0.00      0.00       137
          Syn       1.00      0.99      0.99      9875
         TFTP       1.00      0.99      1.00     19783
          UDP       0.73      0.00      0.

/home/greg/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7280988478568282
               precision    recall  f1-score   support

       Benign       0.91      0.71      0.80     19566
    DrDoS_DNS       0.00      0.00      0.00       734
   DrDoS_LDAP       0.00      0.00      0.00       288
  DrDoS_MSSQL       0.00      0.00      0.00      1243
    DrDoS_NTP       0.82      0.78      0.80     24273
DrDoS_NetBIOS       0.00      0.00      0.00       120
   DrDoS_SNMP       1.00      0.00      0.00       543
    DrDoS_UDP       0.00      0.00      0.00      2084
         LDAP       1.00      0.00      0.00       382
        MSSQL       0.29      0.99      0.45      1704
      NetBIOS       0.00      0.00      0.00       128
      Portmap       0.00      0.01      0.01       137
          Syn       1.00      0.76      0.87      9875
         TFTP       0.84      0.85      0.84     19783
          UDP       0.37      0.77      0.50      3618
      UDP-lag       0.38      0.70      0.49      1786
      WebDDoS       0.00      0.60 